In [71]:
import pandas as pd
import numpy as np
import geopandas as gpd
import fiona

In [72]:
# // get geography files
warnCounties = pd.read_excel('data/warnLogs_200720.xlsx',
                             sheet_name="warnCounties",
                             converters={'FIPS':str})

warnCities= pd.read_excel('data/warnLogs_200720.xlsx',
                          sheet_name="warnCities",
                          usecols="A,B,E,F")

msa = pd.read_excel('../data/geographies/cbsa.xlsx',
                    sheet_name="cbsa",
                    usecols="A,D,M",
                    converters={'FIPS_COUNTY':str,
                                'CBSA Code':str})

msa.rename(columns={'CBSA Code':'CBSA','CBSA Title':'MSA','FIPS_COUNTY':'FIPS'}, inplace=True)

zipWalk = pd.read_excel('../data/spatial/zips/Zip_to_zcta_crosswalk_2020.xlsx',
                        sheet_name='ziptozcta2019',
                        usecols='A,E',
                        converters={'ZIP_CODE':str,
                                    'ZCTA':str})

In [73]:
data = pd.read_excel('data/warnLogs_200720.xlsx',
                     sheet_name="warnLogs",
                     usecols="A:F,H",
                     converters={'ZIP':str})

data['Companies'] = 1

# trim all columns
data['Company name'] = data['Company name'].str.strip()
data['City'] = data['City'].str.strip()
data['ZIP'] = data['ZIP'].str.strip()
data['County'] = data['County'].str.strip()

data.rename(columns={'County':'warnCounty',
                     'City':'warnCity',
                     'Company name':'Company',
                     'Est. Impact':'Employees'}, inplace=True)

In [74]:
data = pd.merge(data, warnCounties, how='left', on='warnCounty')
data = pd.merge(data, warnCities, how='left', on='warnCity')
data = pd.merge(data, msa, how='left', on='FIPS')

In [75]:
data = pd.merge(data, zipWalk, how='left', left_on='ZIP', right_on='ZIP_CODE')

In [76]:
data = data[['Date','Company','Companies','Employees','City',
             'ZIP','County','FIPS','MSA','CBSA','ZCTA']]

In [77]:
data.sort_values('Date', inplace=True)

In [78]:
data.reset_index(drop=True, inplace=True)

In [79]:
data['Month'] = data['Date'].dt.strftime('%b')
data['Year'] = data['Date'].dt.strftime('%Y')

In [80]:
data['Date'] = pd.to_datetime(data['Date'], format='%Y-%m-%d').dt.strftime('%m-%d-%Y')

In [81]:
counties = gpd.read_file('../data/spatial/ga-custom/ga-counties.geojson')
counties = counties[['GEOID','CDRegion']]

In [82]:
data = pd.merge(data,counties, how='left', left_on='FIPS', right_on='GEOID')

In [83]:
data = data[['Date','Month','Year',
             'Company','City','ZIP','County','FIPS','MSA','CBSA','ZCTA','CDRegion',
             'Companies','Employees']]

In [84]:
data.head()

,Date,Month,Year,Company,City,ZIP,County,FIPS,MSA,CBSA,ZCTA,CDRegion,Companies,Employees
0,01-04-2007,Jan,2007,"Polymer Group, Inc",Gainesville,30504,Hall County,13139,"Gainesville, GA",23580,30504,Northeast,1,58
1,01-08-2007,Jan,2007,Enterprise Fund Distributors,Atlanta,30326,Fulton County,13121,"Atlanta-Sandy Springs-Alpharetta, GA",12060,30326,Metro West,1,89
2,01-08-2007,Jan,2007,Marriott Atlanta Global Reservations Sales & C...,Sandy Springs,30328,Fulton County,13121,"Atlanta-Sandy Springs-Alpharetta, GA",12060,30328,Metro West,1,153
3,01-08-2007,Jan,2007,"Glen Raven Custom Fabrics,llc",Elberton,30635,Elbert County,13105,NaN,NaN,30635,Northeast,1,54
4,01-08-2007,Jan,2007,Schwan's Bakery Inc.,Suwanee,30024,Gwinnett County,13135,"Atlanta-Sandy Springs-Alpharetta, GA",12060,30024,Metro North,1,200


In [85]:
# write files
data.to_csv('application/data/warnLogs.csv', index=False)
# zctaRoll.to_csv('application/data/zctaRoll.csv', index=False)
# cityRoll.to_csv('application/data/cityRoll.csv', index=False)

In [86]:
dataJson = data.copy()

In [87]:
zctaRoll = data.groupby(['ZCTA','Year']).agg({'Employees':sum}).reset_index()
zctaRoll = zctaRoll.pivot_table('Employees', ['ZCTA'], 'Year')
zctaRoll.reset_index( drop=False, inplace=True )

In [88]:
columns = ['ZCTA','Employees2007','Employees2008','Employees2009','Employees2010','Employees2011',
           'Employees2012','Employees2013','Employees2014','Employees2015','Employees2016','Employees2017',
           'Employees2018','Employees2019','Employees2020']
zctaRoll.columns = columns
numCols = ['Employees2007','Employees2008','Employees2009','Employees2010','Employees2011',
           'Employees2012','Employees2013','Employees2014','Employees2015','Employees2016','Employees2017',
           'Employees2018','Employees2019','Employees2020']
zctaRoll.fillna(0, inplace=True)

In [89]:
for col in numCols:
    zctaRoll[col] = zctaRoll[col].astype(int)

In [90]:
gaZips = gpd.read_file('../data/spatial/ga-custom/gaZips.geojson')

In [91]:
gaZips['ZCTA'] = gaZips['ZCTA'].astype(str)
zctaRoll['ZCTA'] = zctaRoll['ZCTA'].astype(str)

In [92]:
gaZips = pd.merge(gaZips, zctaRoll, how='left', on='ZCTA')

In [93]:
gaZips.fillna(0, inplace=True)
for col in numCols:
    gaZips[col] = gaZips[col].astype(int)

In [94]:
gaZips = gaZips[['ZCTA','CDRegion','MSA','County',
                 'Employees2007','Employees2008','Employees2009','Employees2010','Employees2011',
                 'Employees2012','Employees2013','Employees2014','Employees2015','Employees2016',
                 'Employees2017','Employees2018','Employees2019','Employees2020','geometry']]

In [95]:
gaZips.to_file('application/data/gaZips.geojson', driver='GeoJSON')

In [96]:
dataJson.to_json('application/data/warnLogs.json', orient='records')

In [97]:
warnCities = pd.read_csv('data/warnCitiesFinal.csv')

In [99]:
cityRoll.head()

,City,Companies2007,Companies2008,Companies2009,Companies2010,Companies2011,Companies2012,Companies2013,Companies2014,Companies2015,Companies2016,Companies2017,Companies2018,Companies2019,Companies2020
0,Acworth,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
1,Adairsville,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
2,Adel,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Ailey,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,Albany,1.0,0.0,5.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,5.0


In [101]:
cityRoll = data.groupby(['City','Year']).agg({'Companies':sum}).reset_index()
cityRoll = cityRoll.pivot_table('Companies', ['City'], 'Year')
cityRoll.reset_index( drop=False, inplace=True )

columns = ['City','Companies2007','Companies2008','Companies2009','Companies2010','Companies2011',
           'Companies2012','Companies2013','Companies2014','Companies2015','Companies2016','Companies2017',
           'Companies2018','Companies2019','Companies2020']
cityRoll.columns = columns
numCols = ['Companies2007','Companies2008','Companies2009','Companies2010','Companies2011',
           'Companies2012','Companies2013','Companies2014','Companies2015','Companies2016','Companies2017',
           'Companies2018','Companies2019','Companies2020']
cityRoll.fillna(0, inplace=True)

for col in numCols:
    cityRoll[col] = cityRoll[col].astype(int)

In [102]:
warnCities = pd.merge(warnCities, cityRoll, how='left', on='City')

In [103]:
warnCities.fillna(0, inplace=True)
for col in numCols:
    warnCities[col] = warnCities[col].astype(int)

In [104]:
warnCities = warnCities[['City','CDRegion','MSA','County',
                         'Companies2007','Companies2008','Companies2009','Companies2010','Companies2011',
                         'Companies2012','Companies2013','Companies2014','Companies2015','Companies2016',
                         'Companies2017','Companies2018','Companies2019','Companies2020',
                         'Latitude','Longitude']]

In [106]:
warnCities.to_csv('data/warnCitiesFinal.csv', index=False)

In [107]:
warnCityTest = warnCities.loc[warnCities['CDRegion'] == 'Metro South']